In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
customers_df = pd.read_csv("Customers.csv", dtype={"CustomerID": str})  # Ensuring CustomerID is string
transactions_df = pd.read_csv("Transactions.csv", dtype={"CustomerID": str})
products_df = pd.read_csv("Products.csv")

In [19]:
merged_df = transactions_df.merge(customers_df, on="CustomerID").merge(products_df, on="ProductID")

In [20]:
customer_summary = merged_df.groupby("CustomerID").agg(
    TotalSpend=("TotalValue", "sum"),
    AvgTransactionValue=("TotalValue", "mean"),
    TotalTransactions=("TransactionID", "count"),
    LastPurchase=("TransactionDate", "max")
).reset_index()

customer_summary = customer_summary.merge(customers_df, on="CustomerID")
customer_summary["LastPurchase"] = pd.to_datetime(customer_summary["LastPurchase"], errors="coerce")
customer_summary["SignupDate"] = pd.to_datetime(customer_summary["SignupDate"], errors="coerce")


In [21]:
customer_summary["LastPurchase"].fillna(pd.Timestamp("2000-01-01"), inplace=True)
customer_summary["SignupDate"].fillna(pd.Timestamp("2000-01-01"), inplace=True)
customer_summary["Recency"] = (pd.Timestamp.today() - customer_summary["LastPurchase"]).dt.days
customer_summary["Tenure"] = (pd.Timestamp.today() - customer_summary["SignupDate"]).dt.days
customer_summary.drop(columns=["LastPurchase", "SignupDate"], inplace=True)

C:\Users\K NANDA KUMAR REDDY\AppData\Local\Temp\ipykernel_14604\3920064164.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_summary["LastPurchase"].fillna(pd.Timestamp("2000-01-01"), inplace=True)
C:\Users\K NANDA KUMAR REDDY\AppData\Local\Temp\ipykernel_14604\3920064164.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on w

In [22]:
if "Region" in customer_summary.columns:
    customer_summary = pd.get_dummies(customer_summary, columns=["Region"], drop_first=True)
scaler = StandardScaler()
features = ["TotalSpend", "AvgTransactionValue", "TotalTransactions", "Recency", "Tenure"]
customer_summary[features] = scaler.fit_transform(customer_summary[features])
similarity_matrix = cosine_similarity(customer_summary[features])

In [23]:
def recommend_similar_customers(customer_id, n=3):
    if customer_id not in customer_summary["CustomerID"].values:
        return "Customer not found!"
    index = customer_summary[customer_summary["CustomerID"] == customer_id].index[0]
    similarity_scores = list(enumerate(similarity_matrix[index]))


In [24]:
sorted_customers = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:n+1]

NameError: name 'similarity_scores' is not defined

In [25]:
  recommendations = [
        {"CustomerID": customer_summary.iloc[i[0]]["CustomerID"], "SimilarityScore": round(i[1], 4)}
        for i in sorted_customers
    ]
    return recommendations


IndentationError: unexpected indent (2972723528.py, line 6)

In [26]:
similar_customers = recommend_similar_customers("C001", n=3)
print(similar_customers)

Customer not found!
